# Kopp et al 2021 Training 

**Authorship:**
Adam Klie, *08/07/2022*
***
**Description:**
This notebook is a use case in EUGENe paper. It is used to showcase Janggu integrated functionality to classify junD binding sites.
***

In [1]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload 
%autoreload 2

import os
import logging
import torch
import numpy as np
import pandas as pd
import eugene as eu

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: Quadro RTX 5000


In [2]:
# Configure EUGENe 
eu.settings.dataset_dir = "/cellar/users/aklie/data/eugene/kopp21"
eu.settings.output_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/output/kopp21"
eu.settings.logging_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/logs/kopp21"
eu.settings.config_dir = "/cellar/users/aklie/projects/EUGENe/EUGENe_paper/configs/kopp21"
eu.settings.verbosity = logging.ERROR

# Load in the `SeqData`

In [6]:
sdata = eu.dl.read_h5sd(
    filename=os.path.join(eu.settings.dataset_dir, "jund_train_processed.h5sd"),
)
sdata

SeqData object with = 948771 seqs
seqs = (948771,)
names = (948771,)
rev_seqs = None
ohe_seqs = (948771, 4, 500)
ohe_rev_seqs = (948771, 4, 500)
seqs_annot: 'chr', 'end', 'start', 'target', 'train_test', 'train_val'
pos_annot: None
seqsm: None
uns: None

In [8]:
sdata = sdata[sdata["chr"] == "chr1"]

In [14]:
sdata = sdata[sdata["end"] <= 1011250]

In [17]:
sdata.seqs_annot[sdata.seqs_annot["target"] == 1]

,chr,end,start,target,train_test,train_val
chr1:1000450-1000950,chr1,1000950,1000450,1.0,True,True
chr1:1000650-1001150,chr1,1001150,1000650,1.0,True,True
chr1:1000850-1001350,chr1,1001350,1000850,1.0,True,True


# Model initialization 

In [4]:
from pytorch_lightning import seed_everything
def prep_new_model(
    seed,
    arch,
    config
):
    # Instantiate the model
    model = eu.models.load_config(
        arch=arch,
        model_config=config
    )

    # Set a seed
    seed_everything(seed)
    
    # Initialize the model prior to conv filter initialization
    eu.models.base.init_weights(model)

    # Return the model
    return model 

In [5]:
# Just make sure the model is taking in the proper data
model_types = ["FCN", "CNN", "RNN", "Hybrid", "Kopp21CNN"]
model_names = ["dsFCN", "dsCNN", "dsRNN", "dsHybrid", "Kopp21CNN"]
for model_name, model_type in zip(model_names, model_types):
    print(model_name, model_type)
    model = prep_new_model(0, model_type, os.path.join(eu.settings.config_dir, f"{model_name}.yaml"))
    if model_type == "RNN":
        sdataloader = sdata.to_dataset(transform_kwargs={"transpose": False}).to_dataloader() 
    else:
        sdataloader = sdata.to_dataset(transform_kwargs={"transpose": True}).to_dataloader()
    test_seqs = next(iter(sdataloader))
    print(model(test_seqs[1], test_seqs[2]).size())
    print()

dsFCN FCN


/cellar/users/aklie/opt/miniconda3/envs/eugene_dev/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Global seed set to 0


No transforms given, assuming just need to tensorize).
torch.Size([128, 1])

dsCNN CNN


Global seed set to 0


No transforms given, assuming just need to tensorize).
torch.Size([128, 1])

dsRNN RNN


...

No transforms given, assuming just need to tensorize).
torch.Size([128, 1])

dsHybrid Hybrid


Global seed set to 0


No transforms given, assuming just need to tensorize).
torch.Size([128, 1])

Kopp21CNN Kopp21CNN


Global seed set to 0


No transforms given, assuming just need to tensorize).
torch.Size([128, 1])



In [ ]:
# Train 5 models with 5 different random initializations
model_types = ["FCN", "CNN", "RNN", "Hybrid", "Kopp21CNN"]
model_names = ["dsFCN", "dsCNN", "dsRNN", "dsHybrid", "Kopp21CNN"]
trials = 5
for model_name, model_type in zip(model_names, model_types):
    for trial in range(1, trials+1):
        print(f"{model_name} trial {trial}")

        # Initialize the model
        model = prep_new_model(
            arch=model_type, 
            config=os.path.join(eu.settings.config_dir, f"{model_name}.yaml"),
            seed=trial
        )

        if model_type == "RNN":
            t_kwargs = transform_kwargs={"transpose": False}
        else:
            t_kwargs = transform_kwargs={"transpose": True}

        # Train the model
        eu.train.fit(
            model=model, 
            sdata=sdata, 
            gpus=1, 
            target="target",
            train_key="train_val",
            epochs=30,
            early_stopping_metric="val_loss",
            early_stopping_patience=5,
            transform_kwargs=t_kwargs,
            batch_size=64,
            num_workers=4,
            name=model_name,
            seed=trial,
            version=f"trial_{trial}",
            verbosity=logging.ERROR
        )
        # Get predictions on the training data
        eu.settings.dl_num_workers = 0
        eu.predict.train_val_predictions(
            model,
            sdata=sdata, 
            target="target",
            train_key="train_val",
            transform_kwargs=t_kwargs,
            name=model_name,
            version=f"trial_{trial}",
            prefix=f"{model_name}_trial_{trial}_"
        )
        del model 
sdata.write_h5sd(os.path.join(eu.settings.output_dir, "train_predictions.h5sd"))

dsFCN trial 1


Global seed set to 1
Global seed set to 1


No transforms given, assuming just need to tensorize).
No transforms given, assuming just need to tensorize).


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | AUROC                     | 0     
1 | fcn       | BasicFullyConnectedModule | 1.1 M 
--------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.232     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

No transforms given, assuming just need to tensorize).
No transforms given, assuming just need to tensorize).


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | AUROC                     | 0     
1 | fcn       | BasicFullyConnectedModule | 1.1 M 
--------------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.232     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 4


Training: 0it [00:00, ?it/s]

---

## Scratch